In [ ]:
import requests

from pathlib import Path
import pandas as pd
# Get the path to the repo root (assuming notebook is in root or subfolder)
notebook_dir = Path().resolve()  # current working directory
# Go up one level to repo root
repo_root = notebook_dir.parent

url = "https://accounts.spotify.com/api/token"

payload = {
    "grant_type": "client_credentials",
    "client_id": "XXXXXXXXXX",
    "client_secret": "XXXXXXXXXX"
}

headers = {
    "Content-Type": "application/x-www-form-urlencoded"
}

In [4]:
# Load data
# Path to CSV
csv_path = repo_root / "data" / "billboard_data_2025_09.csv"

# Load the CSV
df = pd.read_csv(csv_path)
df.head()

,Top Ten Entry Date,Single Name,Artist(s),Peak,Peak Date,Weeks in Top Ten,Ref,Year
0,1960-12-12,Wonderland by Night,Bert Kaempfert,1,1961-01-09,10,NaN,1960
1,1960-12-12,Exodus,Ferrante & Teicher,2,1961-01-23,11,NaN,1960
2,1960-12-26,"Corrina, Corinna",Ray Peterson,9,1961-01-09,5,NaN,1960
3,1960-12-31,Angel Baby,Rosie and the Originals,5,1961-01-23,7,NaN,1960
4,1961-01-09,Will You Love Me Tomorrow,The Shirelles,1,1961-01-30,7,NaN,1961


In [12]:
response = requests.post(url, data=payload, headers=headers)

if response.status_code == 200:
    token_info = response.json()
    access_token = token_info["access_token"]
    print("Access token:", "redacted_for_security")
else:
    print("Error:", response.status_code, response.text)

Access token: redacted_for_security


In [11]:
#Search for the track by title + artist
def search_track(song_title, artist_name):
    search_url = "https://api.spotify.com/v1/search"
    headers = {"Authorization": f"Bearer {access_token}"}
    query = f"track:{song_title} artist:{artist_name}"
    params = {"q": query, "type": "track", "limit": 1}

    response = requests.get(search_url, headers=headers, params=params)
    print(response.json())
    items = response.json().get("tracks", {}).get("items", [])
    
    if items:
        track = items[0]
        track_name = track["name"]
        artist_id = track["artists"][0]["id"]
        return track_name, artist_id
    else:
        return None, None

# Step 3: Get artist genres
def get_artist_genres(artist_id):
    artist_url = f"https://api.spotify.com/v1/artists/{artist_id}"
    headers = {"Authorization": f"Bearer {access_token}"}
    response = requests.get(artist_url, headers=headers)
    print(response.json())
    return response.json().get("genres", [])

# Example usage
song = "Jolene"
artist = "Beyonce"

track_name, artist_id = search_track(song, artist)
if artist_id:
    genres = get_artist_genres(artist_id)
    print(f"Song: {track_name}")
    print(f"Artist genres: {genres}")
else:
    print("Track not found")

{'tracks': {'href': 'https://api.spotify.com/v1/search?offset=0&limit=1&query=track%3AJolene%20artist%3ABeyonce&type=track', 'limit': 1, 'next': 'https://api.spotify.com/v1/search?offset=1&limit=1&query=track%3AJolene%20artist%3ABeyonce&type=track', 'offset': 0, 'previous': None, 'total': 2, 'items': [{'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6vWDO969PvNqNYHIOW5v0m'}, 'href': 'https://api.spotify.com/v1/artists/6vWDO969PvNqNYHIOW5v0m', 'id': '6vWDO969PvNqNYHIOW5v0m', 'name': 'Beyoncé', 'type': 'artist', 'uri': 'spotify:artist:6vWDO969PvNqNYHIOW5v0m'}], 'available_markets': ['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'DE', 'EC', 'EE', 'SV', 'FI', 'FR', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'IE', 'IT', 'LV', 'LT', 'LU', 'MY', 'MT', 'MX', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'SK', 'ES', 'SE', 'CH', 'TW', 'TR', 'UY', 'US', 'GB', 'AD', 'LI', 'MC', 'ID'

In [ ]:
#after testing, it seems spotify's API is unable to reliably find the genres of songs...